In [1]:
import copy
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from math import floor
import logging
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math




In [2]:
from sklearn.metrics import r2_score

In [3]:
from jax import grad, jit, vmap


In [4]:
import jax
import distrax
import blackjax
import jax.numpy as jnp
import flax.linen as nn
import matplotlib.pyplot as plt

from functools import partial
from sklearn.datasets import make_moons
from jax.flatten_util import ravel_pytree


In [5]:
from scipy.io import loadmat
annots = loadmat('/home/paarthsachan/EE499_Project/Dataset/Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW9.mat')
annots_=annots['data'][0][0]
steps=annots_[0][0]
step_different=[]
for i in range(len(steps)):
    if(steps[i][0][0] not in step_different):
        step_different.append(steps[i][0][0])


In [54]:
# battery1_path='/kaggle/input/battery-uniform-distribution-charge-discharge/Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/R/RW9.Rda'
# battery2_path='/kaggle/input/battery-uniform-distribution-charge-discharge/Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/R/RW10.Rda'
# battery3_path='/kaggle/input/battery-uniform-distribution-charge-discharge/Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/R/RW11.Rda'
# battery4_path='/kaggle/input/battery-uniform-distribution-charge-discharge/Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/R/RW12.Rda'
from scipy.io import loadmat
annots = loadmat('/home/paarthsachan/EE499_Project/Dataset/Battery_Uniform_Distribution_Charge_Discharge_DataSet_2Post/data/Matlab/RW12.mat')
annots_=annots['data'][0][0]
steps=annots_[0][0]
dict_type_steps={}
for type_charging in step_different:
    list_filt=[]
    for i in range(100000,len(steps)):
        if(steps[i][0][0]==type_charging):
            list_filt.append(i)
    dict_type_steps[type_charging]=list_filt


In [55]:
dict_type_steps

{'reference charge': [100130,
  100134,
  103149,
  103153,
  106174,
  106178,
  109193,
  109197],
 'reference discharge': [100132,
  100136,
  103151,
  103155,
  106176,
  106180,
  109195,
  109199],
 'pulsed load (rest)': [103157,
  103159,
  103161,
  103163,
  103165,
  103167,
  103169,
  103171,
  109201,
  109203,
  109205,
  109207,
  109209,
  109211,
  109213],
 'pulsed load (discharge)': [103158,
  103160,
  103162,
  103164,
  103166,
  103168,
  103170,
  103172,
  109202,
  109204,
  109206,
  109208,
  109210,
  109212,
  109214],
 'rest (random walk)': [100000,
  100002,
  100004,
  100006,
  100008,
  100010,
  100012,
  100014,
  100016,
  100018,
  100020,
  100022,
  100024,
  100026,
  100028,
  100030,
  100032,
  100034,
  100036,
  100038,
  100040,
  100042,
  100044,
  100046,
  100048,
  100050,
  100052,
  100054,
  100056,
  100058,
  100060,
  100062,
  100064,
  100066,
  100068,
  100070,
  100072,
  100074,
  100076,
  100078,
  100080,
  100082,
  

In [56]:
def array_normaliser(array):
    array_new=(array-array.min()+1e-7)/(array.max()-array.min()+1e-7)
    return array_new
def data_norm(data,select_ele):
    data[:,0]=array_normaliser(data[:,0])
    data[:,1]=array_normaliser(data[:,1])
    data[:,2]=array_normaliser(data[:,2])
    data[:,3]=array_normaliser(data[:,3])
    elements=np.arange(data.shape[0])
    elements_=np.random.choice(elements,select_ele)
    
    #print(data[elements_].shape,"$$$$$$$$$$$$")
    return data[elements_]
def data_norm0(data,select_ele):
    elements=np.arange(data.shape[0])
    elements_=np.random.choice(elements,select_ele)
    
    #print(data[elements_].shape,"$$$$$$$$$$$$")
    return data[elements_]


In [57]:
def norm_give_all_data(steps,step_index):
    step=steps[step_index]
    time_array=step[3][0]

    non_relative_time=step[2][0]
    voltage_array=step[4][0]
    current_array=step[5][0]
    temperature_array=step[6][0]

    voltage_normalised=array_normaliser(voltage_array)
    current_normalised=array_normaliser(current_array)
    nol_rel_normalised=array_normaliser(non_relative_time)
    time_normalised=array_normaliser(time_array)
    temperature_normalised=array_normaliser(temperature_array)
    
    all_data=np.stack((voltage_normalised,current_normalised,time_normalised,temperature_normalised),axis=1)
    
    
    return all_data


In [58]:
def norm_data(data):
    arr0=data[:,0]
    arr1=data[:,1]
    arr2=data[:,2]
    norm_arr0=array_normaliser(arr0)
    norm_arr1=array_normaliser(arr1)
    norm_arr2=array_normaliser(arr2)
    all_data=np.stack((norm_arr0,norm_arr1,norm_arr2),axis=1)

    return all_data

In [59]:


def data_generator(list_filt,index_current,normalised=True,thresh=0):
    
    
            #index_current=100319
    step=steps[list_filt[index_current]]
    time_array=step[3][0]

    non_relative_time=step[2][0]
    voltage_array=step[4][0]
    current_array=step[5][0]
    temperature_array=step[6][0]
    if(normalised==True):
        voltage_normalised=array_normaliser(voltage_array)
        current_normalised=array_normaliser(current_array)
        #nol_rel_normalised=array_normaliser(non_relative_time)
        time_normalised=array_normaliser(time_array)
        temperature_normalised=array_normaliser(temperature_array)
    else:
        voltage_normalised=voltage_array
        current_normalised=current_array
        #nol_rel_normalised=array_normaliser(non_relative_time)
        time_normalised=time_array
        temperature_normalised=temperature_array
    #print(uedjnsm)

    #all_data=np.stack((voltage_array,current_array,time_array,temperature_array),axis=1)
    all_data=np.stack((voltage_normalised,current_normalised,time_normalised,temperature_normalised),axis=1)

    
    total_datapoints=all_data.shape[0]
    
    x1=floor(total_datapoints*0.8)
    x2=floor(total_datapoints*0.8)
    # if((x2-x1>0) and (total_datapoints-x2>0)):
    #     #print(all_data.shape)
    #     #train_data,val_data,test_data=all_data[:x1],all_data[x1:x2],all_data[x2:]## FOr train test val
    #     train_data,test_data=all_data[:x1],all_data[x1:]## For train and test 
    #     return train_data,test_data
    if((x1>thresh)and(total_datapoints-x1>thresh)):
        #print(all_data.shape)
        #train_data,val_data,test_data=all_data[:x1],all_data[x1:x2],all_data[x2:]## FOr train test val
        train_data,test_data=all_data[:x1],all_data[x1:]## For train and test 
        return train_data,test_data
    
    else:
        return None
#             all_error=np.append(all_error,index_current)
#             train_scores=np.append(train_scores,np.nan)
#             validation_scores=np.append(validation_scores,np.nan)
#             test_scores=np.append(test_scores,np.nan)
# #             all_coefficients=np.append(all_coefficients,np.array([np.nan,np.nan,np.nan]))
#             all_intercept=np.append(all_intercept,np.nan)
    #break


    
            

In [60]:
import gc

In [61]:
class MLP1D(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.relu(nn.Dense(3)(x))
        #x = nn.relu(nn.Dense(5)(x))
        x = nn.Dense(1)(x)
        #x=nn.relu(x)
        x=-nn.log_sigmoid(x)
        return x


def bnn_log_joint(params, X, y, model):
    logits = model.apply(params, X).ravel()
    
    flatten_params, _ = ravel_pytree(params)
    log_prior = distrax.Normal(0.0, 1.0).log_prob(flatten_params).sum()
    log_likelihood = distrax.Bernoulli(logits=logits).log_prob(y).sum()
    
    log_joint = log_prior + log_likelihood
    return log_joint


def inference_loop(rng_key, kernel, initial_state, num_samples):
    def one_step(state, rng_key):
        state, _ = kernel(rng_key, state)
        return state, state

    keys = jax.random.split(rng_key, num_samples)
    _, states = jax.lax.scan(one_step, initial_state, keys)

    return states


In [62]:
def likelihood_calculator(pred,target):
    log_likelihood=distrax.Normal(target, 1.0).log_prob(pred).sum()
    return log_likelihood

In [63]:
def bnn_log_joint_rand(params, X, y, model):
    predictions = model.apply(params, X).ravel()
    #print(predictions.shape," :LOGITS SHAPE", y.shape,"  :Target shape ")
    flatten_params, _ = ravel_pytree(params)
    # print(flatten_params.shape,"flatten params shape")
    # print(flatten_params)
    # print("-------------------------------------------------------------------")
    # print(distrax.Normal(1.3027626, 0.1).log_prob(flatten_params))
    
    # print(predictions.shape)
    # print("-------------------------------------------------------------------")
    # print(distrax.Bernoulli(logits=predictions).log_prob(y).shape)# logits is the log odds of 1 event,

    #print(likelihood_calculator(predictions[0],y[0]))
    
    log_prior = distrax.Normal(0.0, 1.0).log_prob(flatten_params).sum()

    log_likelihood_func=vmap(likelihood_calculator,in_axes=(0))
    log_likelihoods=log_likelihood_func(predictions,y)
    #print(log_likelihoods)
    log_likelihood=log_likelihoods.sum()

    
    log_joint = log_prior + log_likelihood
    return log_joint

# num_warmup = 450
# num_steps = 250
# num_samples=15
# X_r=jnp.ones(shape=(30,3))
# y_r=jnp.ones(shape=(30))
# model_rand=MLP1D()
# batch = jnp.ones((num_samples, 3))
# params = model_rand.init(key_init, batch)


# potential_rand=partial(bnn_log_joint_rand, X=X_r, y=y_r, model=model_rand)


In [64]:
def bnn_log_joint_rand2(paramsold,params, X, y, model):
    predictions = model.apply(params, X).ravel()
    print(predictions.shape," :LOGITS SHAPE", y.shape,"  :Target shape ")
    flatten_params, _ = ravel_pytree(params)
    # print(flatten_params.shape,"flatten params shape")
    # print(flatten_params)
    # print("-------------------------------------------------------------------")
    # print(distrax.Normal(1.3027626, 0.1).log_prob(flatten_params))
    
    # print(predictions.shape)
    # print("-------------------------------------------------------------------")
    # print(distrax.Bernoulli(logits=predictions).log_prob(y).shape)# logits is the log odds of 1 event,

    print(likelihood_calculator(predictions[0],y[0]))
    
    log_prior = distrax.Normal(0.0, 1.0).log_prob(flatten_params).sum()

    log_likelihood_func=vmap(likelihood_calculator,in_axes=(0))
    log_likelihoods=log_likelihood_func(predictions,y)
    print(log_likelihoods)
    log_likelihood=log_likelihoods.sum()

    
    log_joint = log_prior + log_likelihood
    return log_joint

# num_warmup = 450
# num_steps = 250
# num_samples=15
# X_r=jnp.ones(shape=(30,3))
# y_r=jnp.ones(shape=(30))
# model_rand=MLP1D()
# batch = jnp.ones((num_samples, 3))
# params = model_rand.init(key_init, batch)
# potential_rand=partial(bnn_log_joint_rand2, X=X_r, y=y_r, model=model_rand)
# potential_rand2=bnn_log_joint_rand2(X=X_r, y=y_r, model=model_rand)
# batch = jnp.ones((num_samples, 3))
# adapt = blackjax.window_adaptation(blackjax.nuts, potential_rand(), num_warmup)

# final_state, kernel, _ = adapt.run(key_warmup, params)
# states = inference_loop(key_samples, kernel, final_state, num_steps)
# sampled_params = states.position



In [65]:
key = jax.random.PRNGKey(314)
key_samples, key_init, key_warmup, key = jax.random.split(key, 4)


In [66]:
import warnings
warnings.filterwarnings('ignore')


In [67]:
median_train_scores=[]
median_test_scores=[]
median_train_std_scores=[]
median_test_std_scores=[]
step_type_array=[]
data_points_arr=[]
for type_step , list_filt in dict_type_steps.items():
    test_scores=[]
    train_scores=[]
    train_std=[]
    test_std=[]
    #mmd_scores=[]
    num_warmup = 80
    num_steps = 20
    num_samples=50
    #print(jd)
    size_limit=20
    if(len(list_filt)>20):
        skip_step=len(list_filt)//20
    else:
        skip_step=1
#     print(list_filt)
    list_filt_skipped=list_filt[::skip_step]
#     print(list_filt_skipped)
    for i in tqdm(range(len(list_filt_skipped))):
        index_current=i
        Data=data_generator(list_filt_skipped,index_current,normalised=False,thresh=200)
        if(Data!=None):
            train_data,test_data=Data
            train_data[:,:-1]=norm_data(train_data[:,:-1])
            train_X,train_y=train_data[:,:-1],train_data[:,-1]
            #train_y=array_normaliser(train_y)
            #print(test_data.shape,"###############")

            test_data[:,:-1]=norm_data(test_data[:,:-1])
            test_X,test_y=test_data[:,:-1],test_data[:,-1]
            #test_data=array_normaliser(test_data)
            
            model = MLP1D()
            potential = partial(bnn_log_joint_rand, X=train_X, y=train_y, model=model)

            batch = jnp.ones((num_samples, 3))
            params = model.init(key_init, batch)
            #print(train_X.shape,train_y.shape," train shapes #####")
            #print(test_X.shape,test_y.shape," test shapes #####")
            
            adapt = blackjax.window_adaptation(blackjax.nuts, potential, num_warmup)## Potential is the log prob sum , could easily put custom here  

            final_state, kernel, _ = adapt.run(key_warmup, params)
            states = inference_loop(key_samples, kernel, final_state, num_steps)
            sampled_params = states.position
            vapply = jax.vmap(model.apply, in_axes=(0, None), out_axes=0)
            #vapply = jax.vmap(vapply, in_axes=(None, 1), out_axes=1)
            #vapply = jax.vmap(vapply, in_axes=(None, 2), out_axes=2)

            outputs = vapply(sampled_params, test_X)
            outputs_train=vapply(sampled_params, train_X)
            # print(train_y[0],"Train first")
            # print(outputs_train.shape,"OUTPUT TRAIN SHAPE")
            outputs_train=jnp.squeeze(outputs_train)
            outputs_train_mean=jnp.mean(outputs_train,axis=0)
            std_output_train=np.std(outputs_train,axis=0)#/outputs_train
            mean_std_train=np.mean(std_output_train)

            outputs_test=jnp.squeeze(outputs)
            outputs_test_mean=jnp.mean(outputs_test,axis=0)
            std_output_test=np.std(outputs_test,axis=0)#/outputs_test_mean
            mean_std_test=np.mean(std_output_test)

            test_r2_score=r2_score(test_y, outputs_test_mean)
            train_r2_score=r2_score(train_y, outputs_train_mean)
            
            
            train_scores.append(train_r2_score)
            test_scores.append(test_r2_score)
            train_std.append(mean_std_train)
            test_std.append(mean_std_test)
            if(mean_std_train==np.nan or mean_std_test==np.nan):
                print(jd)
            # print(std_output_test.shape," STD OF TEST OUTPUTS")
            # print(mean_std_train," MEAN STD OF TRAIN OUTPUTS",mean_std_test," MEAN STD OF TEST OUTPUTS")

            # #print(outputs_test_mean.shape,test_y.shape,"  TEST")
            # #print(outputs_train_mean.shape,train_y.shape,"  TRAIN")
            # print(outputs_test.shape,"PLAIN OUTPUTS")

            # print(r2_score(test_y, outputs_test_mean), "       R2 Test score")
            # print(r2_score(train_y, outputs_train_mean), "       R2 Train score")


            # print("MEAN SHAPE#######",outputs_train_mean.shape)
            #arr_outputs=
            # print("MEAAN",np.mean(outputs_train[0][0]))
            # plt.subplot(1,2,1)
            # plt.plot(outputs_train_mean,color='r')
            # plt.plot(train_y,color='g')
            # plt.title("TRAIN_FIT")
            # plt.subplot(1,2,2)
            # plt.plot(outputs_test_mean,color='r')
            # plt.plot(test_y,color='g')
            # plt.title("TEST FIT")


            # plt.show()

            #print(jd)
            # print(train_X)
            # print(train_y,"#############")
            #print(outputs.shape, " :Outputs",train_X.shape," :Test_X")
            #print(sampled_params['params']," sampled params ")
            del(sampled_params)
            del(states)
            del(outputs)
            del(potential)
            del(model)
            gc.collect()
        
            
        else:
            pass
    test_scores=np.array(test_scores)
    train_scores=np.array(train_scores)
    train_std=np.array(train_std)
    test_std=np.array(test_std)


    median_train_scores.append(np.median(train_scores))   
    median_test_scores.append(np.median(test_scores)) 

    median_train_std_scores.append(np.median(train_std))   
    median_test_std_scores.append(np.median(test_std)) 

    
    
    step_type_array.append(type_step)
    #print(test_scores.shape[0],train_scores.shape[0],"###############")
    data_points_arr.append(test_scores.shape[0]) 
    

 20%|██        | 2/10 [00:32<02:10, 16.35s/it]

In [ ]:
df_stats_shift=pd.DataFrame({"Step_type":step_type_array,"Median_train":median_train_scores,"Median_test":median_test_scores,"Number_datapoints":data_points_arr,
                             "Std Train":median_train_std_scores,"Std Test":median_test_std_scores })

In [ ]:
df_stats_shift

,Step_type,Median_train,Median_test,Number_datapoints,Std Train,Std Test
0,reference charge,0.984349,-5083.256697,5,0.059494,0.810359
1,reference discharge,NaN,NaN,0,NaN,NaN
2,pulsed load (rest),0.992789,-212.182920,14,0.050927,0.176015
3,pulsed load (discharge),NaN,NaN,0,NaN,NaN
4,rest (random walk),NaN,NaN,0,NaN,NaN
5,discharge (random walk),NaN,NaN,0,NaN,NaN
6,charge (random walk),NaN,NaN,0,NaN,NaN
7,pulsed charge (rest),0.986170,-187.615593,4,0.043957,0.152363
8,pulsed charge (charge),NaN,NaN,0,NaN,NaN
9,rest prior low current discharge,NaN,NaN,0,NaN,NaN
